In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Beginner's Level Notebook 
## The following notebook is written only to practice the implementation of ANN on a basic dataset. 
## Basic hyperparameter optimisation is discussed.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
%matplotlib inline

In [ ]:
data = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")


In [ ]:
data.head(10)

In [ ]:
data.dtypes

In [ ]:
data["Outcome"].value_counts()

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize=(7, 5))
sns.countplot(data=data, x='Outcome')
plt.show()

In [ ]:
data.shape


In [ ]:
# check if any null value is present
data.isnull().values.any()

In [ ]:
import seaborn as sns
#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True)

In [ ]:
data.corr()

In [ ]:
Y=data["Outcome"]
Y

In [ ]:
X=data.drop("Outcome",axis=1)

In [ ]:
X.head(10)

In [ ]:
X=X.astype(float)

In [ ]:
X.dtypes

In [ ]:
X.head(10)

In [ ]:
lst=["Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction"]

In [ ]:
for i in lst:
  tem=X[i]
  mean=X[i].mean()
  for j in range(len(tem)):
    if tem[j]==0:
      tem[j]=mean
  X.replace(i,tem,inplace=True)

  
  

In [ ]:
X.head(10)

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(32,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(X, Y,validation_split=0.33, epochs=300)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')

As we can see epochs less than 300 are sufficient

NOW LETS NORMALISE OUR DATASET ANT THEN FEED IT INTO OUR MODEL

In [ ]:
from sklearn import preprocessing

x = X.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_transformed = pd.DataFrame(x_scaled)

In [ ]:
model_2 = Sequential()
model_2.add(Dense(64, input_dim=8, activation='relu',kernel_initializer='glorot_uniform'))
model_2.add(Dense(32,activation='relu',kernel_initializer='glorot_uniform'))
model_2.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
model_2.add(Dense(1, activation='sigmoid'))
# Compile model
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history_2=model_2.fit(x_transformed, Y,validation_split=0.33, epochs=100)

In [ ]:
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')

We can clearly see that after noramlising our dataset we need much less epochs (less than 50) and we also get much higher accuracy. 

INTRODUCING BATCH SIZE  

In [ ]:
model_3 = Sequential()
model_3.add(Dense(64, input_dim=8, activation='relu',kernel_initializer='glorot_uniform'))
model_3.add(Dense(32,activation='relu',kernel_initializer='glorot_uniform'))
model_3.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
model_3.add(Dense(1, activation='sigmoid'))
# Compile model
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history_3=model_3.fit(x_transformed, Y,validation_split=0.33, epochs=100, batch_size=20)

In [ ]:
plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')

As we can see here that using batch propagation is not much helpful in small datasets